<a href="https://colab.research.google.com/github/manishkadam007/HackerEarth-Deep-Learning-challenge-FriendshipGoals/blob/master/Hackerearth_Friendshipday_DL_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Gen RAM Free: 12.7 GB  | Proc size: 112.0 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!cp "/content/drive/My Drive/Train_data" -r "/content/"

In [5]:
!ls

drive  sample_data  Train_data


In [7]:
import torch
import torchvision.models as models
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import os
from torchvision import datasets
import torch.nn as nn
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from torch.utils.data.sampler import SubsetRandomSampler

In [8]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ColorJitter(hue=.05, saturation=.05),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [9]:
train_data = datasets.ImageFolder('Train_data', transform = train_transform)

In [10]:
len(train_data)

3000

In [11]:
from torch.utils.data.sampler import SubsetRandomSampler

In [12]:
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2

num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

In [13]:
len(train_idx)

2400

In [14]:
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [16]:
# number of subprocesses to use for data loading
num_workers = 0

In [17]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)

In [18]:
len(train_loader)*batch_size

2400

In [19]:
len(valid_loader)*batch_size

600

In [20]:
loaders = {}
loaders['train'] = train_loader
loaders['valid'] = valid_loader

In [21]:
model_transfer = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [54]:
use_cuda = torch.cuda.is_available()

In [55]:
use_cuda

True

In [31]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        correct_train = 0.
        total_train = 0.
        
        correct_valid = 0.
        total_valid = 0.
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)
            pred = output.data.max(1, keepdim=True)[1]
            # compare predictions to true label
            correct_train += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total_train += data.size(0)
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
            pred = output.data.max(1, keepdim=True)[1]
            # compare predictions to true label
            correct_valid += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total_valid += data.size(0)
            
            
        train_loss = train_loss/(len(train_loader)*batch_size)
        valid_loss = valid_loss/(len(valid_loader)*batch_size)
        
        train_accuracy = 100 * correct_train / total_train
        valid_accuracy = 100 * correct_valid / total_valid
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tTraining Accuracy: {:.2f} \tValidation Accuracy: {:.2f}'.format(
            epoch, 
            train_loss,
            valid_loss,
            train_accuracy,
            valid_accuracy
            ))
        
        ## TODO: save the model if validation loss has decreased
        
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
        
    # return trained model
    return model

In [32]:
for param in model_transfer.parameters():
    param.requires_grad = False

In [33]:
model_transfer.fc = nn.Linear(2048,3)

In [34]:
if use_cuda:
    model_transfer = model_transfer.cuda()

In [35]:
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = torch.optim.Adam(model_transfer.fc.parameters())

In [36]:
train_data.classes

['adults', 'teenagers', 'toddlers']

In [37]:
# train the model
model_transfer = train(20, loaders, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

Epoch: 1 	Training Loss: 0.767243 	Validation Loss: 0.554422 	Training Accuracy: 66.58 	Validation Accuracy: 77.50
Validation loss decreased (inf --> 0.554422).  Saving model ...
Epoch: 2 	Training Loss: 0.565089 	Validation Loss: 0.445419 	Training Accuracy: 77.00 	Validation Accuracy: 85.33
Validation loss decreased (0.554422 --> 0.445419).  Saving model ...
Epoch: 3 	Training Loss: 0.461418 	Validation Loss: 0.411044 	Training Accuracy: 83.04 	Validation Accuracy: 85.83
Validation loss decreased (0.445419 --> 0.411044).  Saving model ...
Epoch: 4 	Training Loss: 0.449635 	Validation Loss: 0.375351 	Training Accuracy: 82.46 	Validation Accuracy: 87.00
Validation loss decreased (0.411044 --> 0.375351).  Saving model ...
Epoch: 5 	Training Loss: 0.446485 	Validation Loss: 0.355046 	Training Accuracy: 83.12 	Validation Accuracy: 88.83
Validation loss decreased (0.375351 --> 0.355046).  Saving model ...
Epoch: 6 	Training Loss: 0.416835 	Validation Loss: 0.342178 	Training Accuracy: 83.4

In [38]:
# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

<All keys matched successfully>

In [39]:
classes = train_data.classes

In [49]:
def predict_age_group(img_name):
    
    dir_path = "Test Data/Test Data" 
    
    img_path = os.path.join(dir_path, img_name)
    
    img = Image.open(img_path)
    
    img = test_transform(img)
    
    img = torch.unsqueeze(img, 0)

    model_transfer.eval()
    
    img = img.cuda()
    
    out = model_transfer(img) 
    
    _, indices = torch.sort(out, descending=True)

    return classes[indices[0][0]]

In [41]:
import pandas as pd
df = pd.read_csv('Test.csv')

In [42]:
df.head()

,Filename
0,Img1137.jpg
1,Img3633.jpg
2,Img1474.jpg
3,Img3459.jpg
4,Img5578.jpg


In [47]:
!cp "/content/drive/My Drive/Test Data" -r "/content/"

In [50]:
df['Category'] = df['Filename'].apply(predict_age_group)

In [51]:
df['Category'] = df['Category'].map({'adults':'Adults', 'toddlers':'Toddelers', 'teenagers':'Teenagers'})

In [52]:
df.head()

,Filename,Category
0,Img1137.jpg,Adults
1,Img3633.jpg,Teenagers
2,Img1474.jpg,Teenagers
3,Img3459.jpg,Adults
4,Img5578.jpg,Toddelers


In [53]:
df.to_csv('Submission2.csv', index=False)

In [57]:
model_transfer = train(50, loaders, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer2.pt')

Epoch: 1 	Training Loss: 0.315264 	Validation Loss: 0.264517 	Training Accuracy: 88.12 	Validation Accuracy: 91.50
Validation loss decreased (inf --> 0.264517).  Saving model ...
Epoch: 2 	Training Loss: 0.315097 	Validation Loss: 0.366522 	Training Accuracy: 87.54 	Validation Accuracy: 87.50
Epoch: 3 	Training Loss: 0.315593 	Validation Loss: 0.299504 	Training Accuracy: 88.33 	Validation Accuracy: 88.83
Epoch: 4 	Training Loss: 0.302910 	Validation Loss: 0.305302 	Training Accuracy: 88.58 	Validation Accuracy: 87.33
Epoch: 5 	Training Loss: 0.328575 	Validation Loss: 0.276810 	Training Accuracy: 87.29 	Validation Accuracy: 90.33
Epoch: 6 	Training Loss: 0.302834 	Validation Loss: 0.291579 	Training Accuracy: 88.92 	Validation Accuracy: 90.33
Epoch: 7 	Training Loss: 0.299339 	Validation Loss: 0.251314 	Training Accuracy: 88.21 	Validation Accuracy: 91.67
Validation loss decreased (0.264517 --> 0.251314).  Saving model ...
Epoch: 8 	Training Loss: 0.306644 	Validation Loss: 0.231547 	

In [59]:
# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer2.pt'))
import pandas as pd
df = pd.read_csv('Test.csv')
df['Category'] = df['Filename'].apply(predict_age_group)
df['Category'] = df['Category'].map({'adults':'Adults', 'toddlers':'Toddelers', 'teenagers':'Teenagers'})
df.to_csv('Submission3.csv', index=False)

In [62]:
# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        self.conv1 = nn.Conv2d(3,16,3, padding = 1)
        self.conv2 = nn.Conv2d(16,32,3, padding = 1)
        self.conv3 = nn.Conv2d(32,64,3, padding = 1)
        self.conv4 = nn.Conv2d(64,128,3, padding = 1)
        self.conv5 = nn.Conv2d(128,256,3, padding = 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 7 * 7, 8000)
        self.fc2 = nn.Linear(8000,4000)
        self.fc3 = nn.Linear(4000,1000)
        self.fc4 = nn.Linear(1000,3)
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        ## Define forward behavior
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = x.view(-1,256*7*7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

#-#-# You so NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()
print(model_scratch)

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12544, out_features=8000, bias=True)
  (fc2): Linear(in_features=8000, out_features=4000, bias=True)
  (fc3): Linear(in_features=4000, out_features=1000, bias=True)
  (fc4): Linear(in_features=1000, out_features=3, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


In [63]:
criterion_scratch = nn.CrossEntropyLoss()
optimizer_scratch = torch.optim.Adam(model_scratch.parameters())

In [65]:
import torch.nn.functional as F

In [67]:
model_scratch = train(50, loaders, model_scratch, optimizer_scratch, criterion_scratch, use_cuda, 'model_scratch.pt')

Epoch: 1 	Training Loss: 1.095929 	Validation Loss: 1.035858 	Training Accuracy: 35.04 	Validation Accuracy: 46.33
Validation loss decreased (inf --> 1.035858).  Saving model ...
Epoch: 2 	Training Loss: 1.043023 	Validation Loss: 0.938850 	Training Accuracy: 45.25 	Validation Accuracy: 55.83
Validation loss decreased (1.035858 --> 0.938850).  Saving model ...
Epoch: 3 	Training Loss: 0.969611 	Validation Loss: 1.064194 	Training Accuracy: 51.96 	Validation Accuracy: 31.17
Epoch: 4 	Training Loss: 0.923113 	Validation Loss: 0.772147 	Training Accuracy: 54.00 	Validation Accuracy: 66.00
Validation loss decreased (0.938850 --> 0.772147).  Saving model ...
Epoch: 5 	Training Loss: 0.782172 	Validation Loss: 0.649998 	Training Accuracy: 64.62 	Validation Accuracy: 71.83
Validation loss decreased (0.772147 --> 0.649998).  Saving model ...
Epoch: 6 	Training Loss: 0.714404 	Validation Loss: 0.610179 	Training Accuracy: 68.67 	Validation Accuracy: 74.00
Validation loss decreased (0.649998 -->

In [68]:
# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

<All keys matched successfully>

In [69]:
df = pd.read_csv('Test.csv')
df['Category'] = df['Filename'].apply(predict_age_group)
df['Category'] = df['Category'].map({'adults':'Adults', 'toddlers':'Toddelers', 'teenagers':'Teenagers'})
df.to_csv('Submission4.csv', index=False)

In [71]:
!cp "/content/model_scratch.pt" -r "/content/drive/My Drive/"

In [72]:
!cp "/content/model_transfer.pt" -r "/content/drive/My Drive/"

In [73]:
!cp "/content/model_transfer2.pt" -r "/content/drive/My Drive/"